In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
data = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/data1.csv', header=None)
print(data)
labels=pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/label1.csv',header=None)
print(labels)

             0         1         2         3         4         5         6   \
0     -0.446891 -0.013397  0.232645  2.156649  1.652923 -0.210531 -0.662227   
1      0.243747 -1.144175 -0.622214 -0.661979 -0.373315  0.520313 -1.307954   
2     -1.417799 -0.088833 -0.647181 -1.141497 -1.321100  0.686798 -1.045569   
3      0.656784  0.381201 -1.039011  1.285315  0.808324  1.428519 -1.144367   
4      1.098520  1.706769 -1.030370  2.001009  2.751551  1.202229 -0.941066   
...         ...       ...       ...       ...       ...       ...       ...   
19995  0.146324 -0.618555  0.547413  0.851301  0.838880 -1.248402  0.698199   
19996 -2.227923 -0.327196  1.258849 -0.945254  3.925262  0.316686 -1.061159   
19997 -0.800405  1.477997  0.090057 -0.973997 -0.204970  0.224539  0.302265   
19998 -1.217344 -0.785401 -0.413781 -0.335324  0.181671 -1.685733 -0.437981   
19999  1.601005 -0.890398 -1.425049  0.887501  0.167205 -0.395042 -0.512554   

             7         8         9         10      

In [23]:
data=data.values
npLabels=labels.values
print(data.shape)
print(npLabels.shape)

(20000, 20)
(20000, 1)


In [24]:
print(np.max(npLabels))
print(np.min(npLabels))

4
0


In [25]:
np.unique(npLabels)

array([0, 1, 2, 3, 4], dtype=int64)

In [26]:
num=np.max(npLabels)+1
oneHot=np.eye(num)[npLabels]
print(oneHot.ndim)
print(oneHot.shape)

3
(20000, 1, 5)


In [27]:
oneHotRe=oneHot.reshape(20000, 5)
print(oneHotRe)

[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [28]:
trainData,testDataDash=train_test_split(data,train_size=0.8,test_size=0.2,shuffle=False)
trainLabel,testLabelDash=train_test_split(oneHotRe,train_size=0.8,test_size=0.2,shuffle=False)

In [29]:
validData,testData=train_test_split(testDataDash,train_size=0.5,test_size=0.5,shuffle=False)
validLabel,testLabel=train_test_split(testLabelDash,train_size=0.5,test_size=0.5,shuffle=False)

In [30]:
h = 15

In [31]:
b1=np.zeros(h)
b2=np.zeros(5)
print(b1)
print(b2)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [32]:
w1=np.random.normal(0, 1, (20,h))
w2=np.random.normal(0, 1, (h,5))

In [33]:
def actFunc(data,choice):

    if(choice==1):
        return np.tanh(data)
    elif(choice==2):
        numerator=np.exp(data)
        return numerator/np.sum(numerator,axis=1,keepdims=True)

In [34]:
alpha = 0.0003

In [35]:
batch=500

In [36]:
def costFunc(y,t):
    return -(t*(np.log(y))+(1-t)*np.log(1-y))

In [37]:
def forwFunc(data):
    a0=data
    z1=np.dot(a0,w1)+b1
    a1=actFunc(z1,1)
    z2=np.dot(a1,w2)+b2
    a2=actFunc(z2,2)
    return a0,z1,a1,z2,a2

In [38]:
def Acc(y,t,size):
    maxData=np.argmax(y,axis=1)
    maxLabel=np.argmax(t,axis=1)
    compare=np.equal(maxData,maxLabel)
    count=np.sum(compare)
    return (count/size)*100

In [39]:
epochs=500

In [40]:
for epoch in range(epochs):
    for iteration in range(int(len(trainData)/batch)):
        a0,z1,a1,z2,a2=forwFunc(trainData[iteration*batch:(iteration+1)*batch,:])
        y=a2
        labelBatch=trainLabel[iteration*batch:(iteration+1)*batch,:]
        del2=(y-labelBatch)
        del1=np.dot(del2,w2.T)*(1 - a1**2)
        dcdw2=np.dot(a1.T,del2)
        dcdw1=np.dot(a0.T,del1)
        dcdb1=np.sum(del1,axis=0)
        dcdb2=np.sum(del2,axis=0)
        w1=w1-alpha*dcdw1
        w2=w2-alpha*dcdw2
        b2=b2-alpha*dcdb2
        b1=b1-alpha*dcdb1
    a0,z1,a1,z2,a2=forwFunc(trainData)
    print("Training Cost",(np.sum(costFunc(a2,trainLabel)))/16000.0)
    print("Training accuracy",Acc(a2,trainLabel,len(trainLabel)))
    
    va0,vz1,va1,vz2,va2=forwFunc(validData)
    vOutput=va2
    accuracy = Acc(vOutput,validLabel,2000.0)
    print("validation accuracy")
    print(accuracy)

Training Cost 2.7221169438628987
Training accuracy 44.48125
validation accuracy
44.7
Training Cost 2.0619312075956544
Training accuracy 55.03125
validation accuracy
55.300000000000004
Training Cost 1.78521997974661
Training accuracy 60.268750000000004
validation accuracy
60.699999999999996
Training Cost 1.6258498410657614
Training accuracy 63.75625
validation accuracy
63.9
Training Cost 1.5198503175700269
Training accuracy 66.4875
validation accuracy
66.85
Training Cost 1.443968328024087
Training accuracy 68.5625
validation accuracy
68.65
Training Cost 1.3863598703046094
Training accuracy 69.875
validation accuracy
70.39999999999999
Training Cost 1.3400611557334892
Training accuracy 71.0375
validation accuracy
71.3
Training Cost 1.3014098155726375
Training accuracy 72.19375
validation accuracy
72.5
Training Cost 1.26786982021374
Training accuracy 73.00625
validation accuracy
73.35000000000001
Training Cost 1.2378538928879061
Training accuracy 73.81875
validation accuracy
74.1
Training 

Training Cost 0.6631274428235343
Training accuracy 87.88125
validation accuracy
86.3
Training Cost 0.6609544240422212
Training accuracy 87.94999999999999
validation accuracy
86.35000000000001
Training Cost 0.6587971241187697
Training accuracy 87.96875
validation accuracy
86.3
Training Cost 0.6566521955963494
Training accuracy 88.0125
validation accuracy
86.35000000000001
Training Cost 0.6545153863155004
Training accuracy 88.125
validation accuracy
86.4
Training Cost 0.6523812724980295
Training accuracy 88.1625
validation accuracy
86.5
Training Cost 0.650243159657145
Training accuracy 88.19375000000001
validation accuracy
86.5
Training Cost 0.6480933188284715
Training accuracy 88.20625
validation accuracy
86.6
Training Cost 0.6459236331839104
Training accuracy 88.28125
validation accuracy
86.7
Training Cost 0.643726531075083
Training accuracy 88.33125
validation accuracy
86.9
Training Cost 0.6414959466590946
Training accuracy 88.35625
validation accuracy
86.9
Training Cost 0.63922813114

validation accuracy
88.6
Training Cost 0.5572787543897899
Training accuracy 90.29374999999999
validation accuracy
88.64999999999999
Training Cost 0.5570652579916299
Training accuracy 90.29374999999999
validation accuracy
88.64999999999999
Training Cost 0.5568545623963195
Training accuracy 90.3
validation accuracy
88.7
Training Cost 0.5566465673428458
Training accuracy 90.30625
validation accuracy
88.7
Training Cost 0.5564411757839084
Training accuracy 90.3125
validation accuracy
88.7
Training Cost 0.5562382936960962
Training accuracy 90.3125
validation accuracy
88.75
Training Cost 0.5560378299134695
Training accuracy 90.3125
validation accuracy
88.75
Training Cost 0.5558396959830462
Training accuracy 90.3
validation accuracy
88.8
Training Cost 0.5556438060406504
Training accuracy 90.30625
validation accuracy
88.8
Training Cost 0.5554500767055426
Training accuracy 90.3
validation accuracy
88.85
Training Cost 0.5552584269922082
Training accuracy 90.30625
validation accuracy
88.85
Trainin

Training Cost 0.5416770038901475
Training accuracy 90.425
validation accuracy
89.3
Training Cost 0.5415326223256514
Training accuracy 90.4125
validation accuracy
89.3
Training Cost 0.5413884650928256
Training accuracy 90.4125
validation accuracy
89.3
Training Cost 0.5412445405322889
Training accuracy 90.4125
validation accuracy
89.3
Training Cost 0.5411008575292066
Training accuracy 90.43124999999999
validation accuracy
89.25
Training Cost 0.54095742558236
Training accuracy 90.44375
validation accuracy
89.25
Training Cost 0.5408142548656976
Training accuracy 90.45625000000001
validation accuracy
89.25
Training Cost 0.5406713562806618
Training accuracy 90.45625000000001
validation accuracy
89.25
Training Cost 0.5405287414977151
Training accuracy 90.4625
validation accuracy
89.25
Training Cost 0.5403864229855885
Training accuracy 90.4625
validation accuracy
89.2
Training Cost 0.5402444140268688
Training accuracy 90.4625
validation accuracy
89.2
Training Cost 0.5401027287186471
Training a

Training Cost 0.5300223121129207
Training accuracy 90.6375
validation accuracy
89.3
Training Cost 0.5299306904314125
Training accuracy 90.64999999999999
validation accuracy
89.3
Training Cost 0.5298396537961767
Training accuracy 90.65625
validation accuracy
89.3
Training Cost 0.5297492066016434
Training accuracy 90.65625
validation accuracy
89.3
Training Cost 0.5296593540320053
Training accuracy 90.64375
validation accuracy
89.3
Training Cost 0.5295701019931957
Training accuracy 90.6375
validation accuracy
89.35
Training Cost 0.5294814570222034
Training accuracy 90.64375
validation accuracy
89.35
Training Cost 0.5293934261763016
Training accuracy 90.6625
validation accuracy
89.35
Training Cost 0.5293060169055886
Training accuracy 90.6625
validation accuracy
89.35
Training Cost 0.5292192369129185
Training accuracy 90.66875
validation accuracy
89.35
Training Cost 0.5291330940057516
Training accuracy 90.66875
validation accuracy
89.35
Training Cost 0.5290475959446761
Training accuracy 90.

Training Cost 0.5237643198817061
Training accuracy 90.8
validation accuracy
89.55
Training Cost 0.5237165525970836
Training accuracy 90.79374999999999
validation accuracy
89.55
Training Cost 0.5236689663963473
Training accuracy 90.79374999999999
validation accuracy
89.55
Training Cost 0.5236215589444545
Training accuracy 90.79374999999999
validation accuracy
89.55
Training Cost 0.5235743280005671
Training accuracy 90.7875
validation accuracy
89.55
Training Cost 0.5235272714152234
Training accuracy 90.7875
validation accuracy
89.55
Training Cost 0.5234803871275464
Training accuracy 90.8
validation accuracy
89.55
Training Cost 0.5234336731624941
Training accuracy 90.80625
validation accuracy
89.55
Training Cost 0.5233871276281432
Training accuracy 90.81875000000001
validation accuracy
89.5
Training Cost 0.5233407487130148
Training accuracy 90.81875000000001
validation accuracy
89.45
Training Cost 0.5232945346834349
Training accuracy 90.825
validation accuracy
89.45
Training Cost 0.523248